Pytorch Attention
- 최신버전엔 flash attention 기능이 들어있음

In [1]:
import torch
import torch.nn.functional as F

In [ ]:
from packaging import version
from functools import partial
torch_version = version.parse(torch.__version__)

print(torch_version)
sdp_kwargs = dict(
        enable_flash = True,
        enable_math = True,
        enable_mem_efficient = True
)

if torch_version >= version.parse('2.3'):
    from torch.nn.attention import SDPBackend
    str_to_backend = dict(
        enable_flash = SDPBackend.FLASH_ATTENTION,
        enable_mem_efficient = SDPBackend.EFFICIENT_ATTENTION,
        enable_math = SDPBackend.MATH,
        enable_cudnn = SDPBackend.CUDNN_ATTENTION
    )

    sdpa_backends = [str_to_backend[enable_str] for enable_str, enable in sdp_kwargs.items() if enable]
    # 최신 버전 : 여러가지 백엔드 활성화를 시킬수 있음
    sdp_context_manager = partial(torch.nn.attention.sdpa_kernel, sdpa_backends)
else: # 폐기 예정
    sdp_context_manager = partial(torch.backends.cuda.sdp_kernel, **sdp_kwargs)

2.5.1+cu124


In [4]:
q = torch.randn(2, 8, 16, 64, device="cuda", dtype=torch.float16)
k = torch.randn(2, 8, 16, 64, device="cuda", dtype=torch.float16)
v = torch.randn(2, 8, 16, 64, device="cuda", dtype=torch.float16)
mask = None

In [10]:
%%timeit
with sdp_context_manager():
    out = F.scaled_dot_product_attention(q, k, v, attn_mask=mask, dropout_p=0.1)

48.9 µs ± 503 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
out.shape

torch.Size([2, 8, 16, 64])

In [11]:
str_to_backend = dict(
    enable_flash = SDPBackend.FLASH_ATTENTION,
    enable_mem_efficient = SDPBackend.EFFICIENT_ATTENTION,
    enable_math = SDPBackend.MATH,
    enable_cudnn = SDPBackend.CUDNN_ATTENTION
)
sdp_context_manager = partial(torch.backends.cuda.sdp_kernel, **sdp_kwargs)


In [12]:
%%timeit
with sdp_context_manager():
    out = F.scaled_dot_product_attention(q, k, v, attn_mask=mask, dropout_p=0.1)

/opt/conda/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


61.3 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Linear attention

- q, k에 kernel 함수를 먼저 적용

![Image](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fcqm9dE%2FbtsshkDxDp5%2F0bouLgFTM4jwSnSr2Ngq40%2Fimg.png)

In [ ]:
LlamaAttention

In [ ]:
class LlamaAttention(nn.Module):
    def __init__(self, config, layer_idx):
        super().__init__()
        self.config = config
        self.layer_idx = layer_idx
        
        self.hidden_size = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = self.hidden_size // self.num_heads
        self.num_key_value_heads = config.num_key_value_heads
        self.num_key_value_groups = self.num_heads // self.num_key_value_heads
        self.max_position_embeddings = config.max_position_embeddings
        self.rope_theta = config.rope_theta
        self.is_causal = True
        

        if (head_dim * num_heads) != hidden_size:
            raise ValueError(f"not divisible")
        
        self.q_proj = nn.Linear(self.hidden_size, self.hidden_size, self.num_heads, bias=config.attention_bias)
        self.k_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=config.attention_bias)
        self.v_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=config.attention_bias)
        self.o_proj = nn.Linear(self.num_heads * self.head_dim, self.hidden_size, bias=config.attention_bias)


        self._init_rope()
        
    def _init_rope(self):
        self.rotary_emb = ...

    def forward(self, hidden_states : torch.Tensor,
                attention_mask: Optional[torch.Tensor] = None,
                position_ids: Optional[torch.LongTensor] = None,
                past_key_value: Optional[Cache] = None,
                output_attentions: bool = False,
                use_cache: bool = False,
                **kwargs,) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:

        bsz, q_len, _ = hidden_states.size()
        